In [28]:
import pandas as pd
import re
import spacy

In [29]:
sp = spacy.load('en_core_web_sm')

In [30]:
x1 = pd.read_csv('../data/sigmod/X1.csv')
x2 = pd.read_csv('../data/sigmod/X2.csv')
x3 = pd.read_csv('../data/sigmod/X3.csv')
x4 = pd.read_csv('../data/sigmod/X4.csv')

In [4]:
len(x1.columns), len(x2.columns), len(x3.columns), len(x4.columns)

(16, 14, 14, 5)

In [5]:
x1.columns # --> convert it to be a validation set 

Index(['cpu_brand', 'battery_chemistry', 'ram_type', 'cpu_frequency',
       'hdd_capacity', 'dimensions_height', 'brand', 'cpu_type', 'title',
       'cpu_cache', 'battery_life', 'dimensions_depth', 'dimensions',
       'display_size', 'instance_id', 'cpu_model'],
      dtype='object')

In [6]:
x2.columns

Index(['instance_id', 'brand', 'cpu_brand', 'cpu_model', 'cpu_type',
       'cpu_frequency', 'ram_capacity', 'ram_type', 'ram_frequency',
       'hdd_capacity', 'ssd_capacity', 'weight', 'dimensions', 'title'],
      dtype='object')

In [7]:
x3.columns

Index(['instance_id', 'brand', 'cpu_brand', 'cpu_model', 'cpu_type',
       'cpu_frequency', 'ram_capacity', 'ram_type', 'ram_frequency',
       'hdd_capacity', 'ssd_capacity', 'weight', 'dimensions', 'title'],
      dtype='object')

In [8]:
x4.columns

Index(['name', 'price', 'brand', 'size', 'instance_id'], dtype='object')

In [9]:
def formatNumber(num):
    num = float(num)
    if num % 1 == 0:
        return int(num)
    else:
        return num
def fill_nulls_with_none(df):
    """ Fills nulls in a dataframe with None.
        This is required for the Dedupe package to work properly.
        
        Input: - dataframe with nulls as NaN
        
        Output: - new dataframe with nulls as None
    """
    new_df = df.copy()
    for col in df.columns:
        new_df[col] = new_df[col].where(new_df[col].notnull(), None)
    return new_df

def convert_numbers_to_strings(df, cols_to_convert, remove_point_zero=True):
    """ Convert number types to strings in a dataframe.
        This is convoluted as need to keep NoneTypes as NoneTypes for what comes next!
        
        Inputs: - df -> dataframe to convert number types
                - cols_to_convert -> list of columns to convert
                - remove_point_zero -> bool to say whether you want '.0' removed from number
        
        Ouputs: - dataframe with converted number types
    """
    new_df = df.copy()
    for col in cols_to_convert:
        if remove_point_zero:
            new_df[col] = new_df[col].apply(lambda x: str(x).replace('.0','')\
                                            if not isinstance(x, type(None)) else x)
        else:
            new_df[col] = new_df[col].apply(lambda x: str(x)\
                                            if not isinstance(x, type(None)) else x)
    return new_df

extra_brands = set(pd.read_csv('laptops.csv').Company.str.lower().unique())
screen_sizes = set(pd.read_csv('laptops.csv').Inches)
screen_sizes = [str(formatNumber(str(s).lower())) for s in screen_sizes]

def preprocess_laptop_dataset(df):
    # Alpha numeric
    irrelevant_regex = re.compile(r'[^a-z0-9,.\-\s]')
    multispace_regex = re.compile(r'\s\s+') # Why it doesn't work
    df.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

    for column in df.columns:
        if column == 'instance_id':
            continue
        df[column] = df[column].str.lower().str.replace(irrelevant_regex, ' ').str.replace(multispace_regex, ' ')
    
    
    def tokenize_new_tile(record):
        return [w.text for w in sp(record['new_title'])]
    
    df['new_title'] = df.title
    irrelevant_regex = re.compile(r'[^a-z0-9.\s]')
    multispace_regex = re.compile(r'\s\s+') # Why it doesn't work
    df['new_title'] = df.new_title.str.lower().str.replace(irrelevant_regex, '').str.replace(multispace_regex, ' ')
    df['new_title_tokens'] =  df.apply(tokenize_new_tile, axis=1) 

    # Brand assignment
    all_brands = set()
    
    all_brands.update(extra_brands)

    def assign_brand(record):
        # Search in brand first
        if record['brand'] in all_brands:
            return record['brand']
        # then in the title
        for el in all_brands:
            if el in record['title']:
                return el
        return "NNN"

    df['brand'] = df.apply(assign_brand, axis=1)

    # cpu brand
    def assign_cpu_brand(record):
        # Search in brand first
        if 'intel' in str(record['cpu_brand']) or 'intel' in str(record['title']) or \
                'intel' in str(record['cpu_model']) or 'intel' in str(record['cpu_type']):
            return 'intel'
        return 'amd'

    df['cpu_brand'] = df.apply(assign_cpu_brand, axis=1)

    def assign_screen_size(record):
        brand_tokens = record['new_title_tokens']
        arr = []
        for t in brand_tokens:
            s = t.replace('inch', '')
            s = s.replace('in', '')
            arr.append(s)
        
        for sc in screen_sizes:
            if str(sc) in arr:
                return str(sc)
        
        else:
            return str(15.6) # Some relaxation
    df['screen_size'] = df.apply(assign_screen_size, axis=1)
    
    # ram capacity
    def assign_ram_capacity(record):
        s = str(record['ram_capacity']).replace(' ', '')
        possible_vals = ['2gb', '4gb', '6gb', '8gb', '10gb', '12gb', '16gb',
                         '32gb', '64gb', '128gb', '256gb', '512gb', '2', '4',
                         '6', '8', '10', '12', '16', '32', '64', '128']
        for val in possible_vals:
            if val in s:
                return int(val.replace('gb', ''))

        s = str(record['title']).replace(' ', '')  # This will be wrong, please change
        possible_vals = ['2gb', '4gb', '6gb', '8gb', '10gb', '12gb', '16gb',
                         '32gb', '64gb', '128gb']
        for val in possible_vals:
            if val in s:
                return int(val.replace('gb', ''))

        return 0
    
    def assign_hdd_capacity(record):
        s = str(record['hdd_capacity']).replace(' ', '')
        s2 = str(record['title'].replace(' ', ''))
        
        if 'ssd' in s:
            return 0

        if re.search("\d{3,4}gb", s):
            return int(re.findall("\d{3,4}gb", s)[0][:-2])
        if re.search("\dtb", s):
            return int(re.findall("\dtb", s)[0][:-2] + '000')
        if re.search("\d{3,4}gbhdd", s2):
            return int(re.findall("\d{3,4}gbhdd", s2)[0][:-5])
        if re.search("hdd\d{3,4}gb", s2):
            return int(re.findall("hdd\d{3,4}gb", s2)[0][3:-2])
        if re.search("hdd\d{1}tb", s2):
            return int(re.findall("hdd\d{1}tb", s2)[0][3:4] + '000')
        if re.search("\d{1}tbhdd", s2):
            return int(re.findall("\d{1}tbhdd", s2)[0][0] + '000')
        return 0
    df['hdd_capacity'] = df.apply(assign_hdd_capacity, axis=1)
                                
    def assign_hdd_capacity(record):
        s = str(record['ssd_capacity']).replace(' ', '')
        s2 = str(record['title'].replace(' ', ''))
        

        if re.search("\d{3,4}gb", s):
            return int(re.findall("\d{3,4}gb", s)[0][:-2])
        if re.search("\dtb", s):
            return int(re.findall("\dtb", s)[0][:-2] + '000')
        if re.search("\d{3,4}gbssd", s2):
            return int(re.findall("\d{3,4}gbssd", s2)[0][:-5])
        if re.search("ssd\d{3,4}gb", s2):
            return int(re.findall("ssd\d{3,4}gb", s2)[0][3:-2])
        if re.search("ssd\d{1}tb", s2):
            return int(re.findall("ssd\d{1}tb", s2)[0][3:4] + '000')
        if re.search("\d{1}tbssd", s2):
            return int(re.findall("\d{1}tbssd", s2)[0][0] + '000')
        return 0
            
    df['ssd_capacity'] = df.apply(assign_hdd_capacity, axis=1)
    
    def assign_laptop_model(record):
        brand_tokens = record['new_title_tokens']
        try:
            brand_index = brand_tokens.index(str(record['brand']))
            finish_index = brand_index + 2
            should_break = False
            for i in range(2 + brand_index, 5 + brand_index, 1):
                for sc in screen_sizes:
                    if (sc in brand_tokens[i]):
                        should_break = True
                        break
                if should_break:
                    if finish_index == i:
                        finish_index -=1
                    break
                if not (brand_tokens[i].isalpha()):
                    finish_index = i 
                else:
                    break
        except:
            brand_index = -1

        if brand_index == -1:
            return None

        return ' '.join(brand_tokens[brand_index+1:finish_index+1])        

    df['model'] = df.apply(assign_laptop_model, axis=1)
    df['ram_capacity'] = df.apply(assign_ram_capacity, axis=1)
    
    df = fill_nulls_with_none(df)
    df = convert_numbers_to_strings(df, ['screen_size'])
    # Unit stand. in weight
    
    def assign_cpu_type(record):
    # Find the cpu type
        cpu_list = ["i5", "i3", "i7", "atom", 
                    "pentium", "celeron", "a-series",
                    "e-series", "aseries", "eseries",
                    "a1", "a2", "a3", "a4", "a5", "a6", "a7", "a8", "a9"]

        for cpu in cpu_list:
            if record['cpu_type'] is not None and cpu in record['cpu_type']:
                return cpu
            if cpu in record['title']:
                return cpu
            if record['cpu_model'] is not None and cpu in record['cpu_model']:
                return cpu
            if record['cpu_frequency'] is not None and  cpu in record['cpu_frequency']:
                return cpu

            if re.search("e-[0-9]{3}", record['title']):
                return re.findall("e-[0-9]{3}", record['title'])[0]

            if record['cpu_model'] is not None and re.search("e-[0-9]{3}", record['cpu_model']):
                return re.findall("e-[0-9]{3}", record['cpu_model'])[0]

    df['cpu_type'] = df.apply(assign_cpu_type, axis=1)
    
    return df

In [614]:
x2 = pd.read_csv('../data/sigmod/X2.csv')
x2.head()

,instance_id,brand,cpu_brand,cpu_model,cpu_type,cpu_frequency,ram_capacity,ram_type,ram_frequency,hdd_capacity,ssd_capacity,weight,dimensions,title
0,www.softwarecity.ca//737,Lenovo,Intel. i5-3320M,i5-3320M,Dual-core ( 2 Core ). Core i5,2.60 GHz,NaN,DDR3 SDRAM. DDR3-1600/PC3-12800. DDR3 SDRAM,DDR3-1600/PC3-12800,320 GB,NaN,1.80 kg,NaN,"Lenovo Thinkpad X230 34352jf Tablet Pc - 12.5""..."
1,www.isupplyhub.com//1256,Acer,1.6 GHz Intel Core i5-4200U. Intel Core I5,NaN,1.6 GHz Intel Core i5-4200U,1.6 GHz Intel Core i5-4200U,8 GB DDR3L SDRAM,DDR3 SDRAM. 8 GB DDR3L SDRAM,NaN,500 GB mechanical_hard_drive,NaN,4.8 pounds,15.02 x 10.08 x 0.90 inches,Amazon.com : Acer Aspire V7-582PG-6479 15.6-In...
2,www.isupplyhub.com//326,Acer,1.6 GHz Intel Core i5. Intel Core I5,NaN,1.6 GHz Intel Core i5,1.6 GHz Intel Core i5,4 GB DDR3-SDRAM,DDR3 SDRAM. 4 GB DDR3-SDRAM,NaN,500 GB mechanical_hard_drive,NaN,5.2 pounds,15.02 x 10.08 x 1 inches,Amazon.com : Acer Aspire E1-572-6870 15.6 Inch...
3,www.isupplyhub.com//821,HP,NaN,NaN,NaN,NaN,4 GB SDRAM DDR3,DDR3 SDRAM. 4 GB SDRAM DDR3,NaN,500 GB,NaN,4.8 pounds,15.18 x 0.89 x 10.16 inches,"Amazon.com : 15.6"" HP 15-f009wm Amd Dual-Core ..."
4,www.isupplyhub.com//157,Asus,1.7 GHz Core i5-3317U. Intel,NaN,1.7 GHz Core i5-3317U,1.7 GHz Core i5-3317U,4 GB DDR3,DDR3 SDRAM. 4 GB DDR3,NaN,256 MB,NaN,2.9 pounds,8.80 x 0.70 x 12.80 inches,Amazon.com : ASUS UX31A-XB52 13.3-Inch Ultrabo...


In [615]:
x = x2.copy()
x.set_index('instance_id',  inplace=True, drop=False)
x = preprocess_laptop_dataset(x)
x = fill_nulls_with_none(x)
x = convert_numbers_to_strings(x, ['ram_capacity', 'screen_size'])
x.head(8)

,instance_id,brand,cpu_brand,cpu_model,cpu_type,cpu_frequency,ram_capacity,ram_type,ram_frequency,hdd_capacity,ssd_capacity,weight,dimensions,title,new_title,new_title_tokens,screen_size,model
instance_id,,,,,,,,,,,,,,,,,,
www.softwarecity.ca//737,www.softwarecity.ca//737,lenovo,intel,i5-3320m,i5,2.60 ghz,4,ddr3 sdram. ddr3-1600 pc3-12800. ddr3 sdram,ddr3-1600 pc3-12800,320,0,1.80 kg,None,lenovo thinkpad x230 34352jf tablet pc - 12.5 ...,lenovo thinkpad x230 34352jf tablet pc 12.5 in...,"[lenovo, thinkpad, x230, 34352jf, tablet, pc, ...",12.5,thinkpad x230 34352jf
www.isupplyhub.com//1256,www.isupplyhub.com//1256,acer,intel,None,i5,1.6 ghz intel core i5-4200u,8,ddr3 sdram. 8 gb ddr3l sdram,None,500,0,4.8 pounds,15.02 x 10.08 x 0.90 inches,amazon.com acer aspire v7-582pg-6479 15.6-inch...,amazon.com acer aspire v7582pg6479 15.6inch to...,"[amazon.com, acer, aspire, v7582pg6479, 15.6in...",15.6,aspire v7582pg6479
www.isupplyhub.com//326,www.isupplyhub.com//326,acer,intel,None,i5,1.6 ghz intel core i5,4,ddr3 sdram. 4 gb ddr3-sdram,None,500,0,5.2 pounds,15.02 x 10.08 x 1 inches,amazon.com acer aspire e1-572-6870 15.6 inch l...,amazon.com acer aspire e15726870 15.6 inch lap...,"[amazon.com, acer, aspire, e15726870, 15.6, in...",15.6,aspire
www.isupplyhub.com//821,www.isupplyhub.com//821,hp,amd,None,None,None,4,ddr3 sdram. 4 gb sdram ddr3,None,500,0,4.8 pounds,15.18 x 0.89 x 10.16 inches,amazon.com 15.6 hp 15-f009wm amd dual-core e1-...,amazon.com 15.6 hp 15f009wm amd dualcore e1210...,"[amazon.com, 15.6, hp, 15f009wm, amd, dualcore...",15.6,15f009wm amd
www.isupplyhub.com//157,www.isupplyhub.com//157,asus,intel,None,i5,1.7 ghz core i5-3317u,4,ddr3 sdram. 4 gb ddr3,None,0,256,2.9 pounds,8.80 x 0.70 x 12.80 inches,amazon.com asus ux31a-xb52 13.3-inch ultrabook...,amazon.com asus ux31axb52 13.3inch ultrabook 1...,"[amazon.com, asus, ux31axb52, 13.3inch, ultrab...",13.3,ux31axb52
www.isupplyhub.com//985,www.isupplyhub.com//985,lenovo,intel,None,i7,intel core i7,0,3 gb,None,160,160,None,None,amazon.com lenovo 3093b51 thinkpad x201 tablet...,amazon.com lenovo 3093b51 thinkpad x201 tablet...,"[amazon.com, lenovo, 3093b51, thinkpad, x201, ...",15.6,3093b51 thinkpad
www.isupplyhub.com//648,www.isupplyhub.com//648,acer,intel,None,i5,2.6 ghz core i5 3230m,6,ddr3 sdram. 6 gb ddr3,None,500,0,7.1 pounds,16.30 x 10.80 x 1.40 inches,amazon.com acer aspire nx.mg7aa.005 e1-771-649...,amazon.com acer aspire nx.mg7aa.005 e17716496 ...,"[amazon.com, acer, aspire, nx.mg7aa.005, e1771...",17.3,aspire nx.mg7aa.005
www.isupplyhub.com//669,www.isupplyhub.com//669,acer,intel,None,i7,None,2,12 gb ddr3,None,1000,120,16 pounds,None,amazon.com acer aspire v3-772g-9460 17.3 led n...,amazon.com acer aspire v3772g9460 17.3 led not...,"[amazon.com, acer, aspire, v3772g9460, 17.3, l...",12,aspire v3772g9460


In [599]:
def assign_hdd_capacity(record):
    s = str(record['hdd_capacity']).replace(' ', '')
    s2 = str(record['title'].replace(' ', ''))
    print(s)
    print(s2)
    print()
    if re.search("\d{3,4}gb", s):
        return re.findall("\d{3,4}gb", s)[0][:-2]
    if re.search("\dtb", s):
        return re.findall("\dtb", s)[0][:-2] + '000'
    if re.search("\d{3,4}gbhdd", s2):
        return re.findall("\d{3,4}gbhdd", s2)[0][:-5]
    if re.search("hdd\d{3,4}gb", s2):
        print (re.findall("hdd\d{3,4}gb", s2)[0])
        return re.findall("hdd\d{3,4}gb", s2)[0][3:-2]
    if re.search("hdd\d{1}tb", s2):
        return re.findall("hdd\d{1}tb", s2)[0][3:4] + '000'
    if re.search("\d{1}tbhdd", s2):
        return re.findall("\d{1}tbhdd", s2)[0][0] + '000'
    return 0

In [600]:
re.findall("\d\d\dgb", str(x.iloc[0, :]['hdd_capacity']).replace(' ', ''))

['320gb']

In [601]:
x.iloc[:6, ].apply(assign_hdd_capacity, axis=1)

320gb
lenovothinkpadx23034352jftabletpc-12.5-in-planeswitchingipstechnology-wirelesslan-intelcorei5i5-3320m2.60ghz-black-4gbram-320gbhdd-windows7professional64-bit-convertible-1366x768multi-touchscreendisplayledbacklight-bluetooth-frenchkeyboard-34352jf-softwarecity.ca-canada

500gbmechanicalharddrive
amazon.comaceraspirev7-582pg-647915.6-inchtouchscreenultrabookcoolsteelcomputersaccessories

500gbmechanicalharddrive
amazon.comaceraspiree1-572-687015.6inchlaptopinteli54200u1.6ghzprocessor4gbram500gbharddrivewindows8clarinetblacklaptopcomputerscomputersaccessories

500gb
amazon.com15.6hp15-f009wmamddual-coree1-21004gbddr3ram500gbhdwebcamwindows8.1certifiedrefurbishedcomputersaccessories

256mb
amazon.comasusux31a-xb5213.3-inchultrabook1.7ghzintelcorei5-3317uprocessor4gbddr3256gbssdwindows7professionalsilveraluminumlaptopcomputerscomputersaccessories

160gb
amazon.comlenovo3093b51thinkpadx201tablet3093-convertible-corei7640lm2.13ghz-winxptabletpc2005-3gbram-160gbssd-noopticaldrive-12.1in

instance_id
www.softwarecity.ca//737    320
www.isupplyhub.com//1256    500
www.isupplyhub.com//326     500
www.isupplyhub.com//821     500
www.isupplyhub.com//157       0
www.isupplyhub.com//985     160
dtype: object

In [416]:
def find_cpu_type(record):
    # Find the cpu type
    cpu_list = ["i5", "i3", "i7", "atom", 
                "pentium", "celeron", "a-series",
                "e-series", "aseries", "eseries",
                "a1", "a2", "a3", "a4", "a5", "a6", "a7", "a8", "a9"]
    
    for cpu in cpu_list:
        if record['cpu_type'] is not None and cpu in record['cpu_type']:
            return cpu
        if cpu in record['title']:
            return cpu
        if record['cpu_model'] is not None and cpu in record['cpu_model']:
            return cpu
        if record['cpu_frequency'] is not None and  cpu in record['cpu_frequency']:
            return cpu
        
        if re.search("e-[0-9]{3}", record['title']):
            return re.findall("e-[0-9]{3}", record['title'])[0]
                
        if record['cpu_model'] is not None and re.search("e-[0-9]{3}", record['cpu_model']):
            return re.findall("e-[0-9]{3}", record['cpu_model'])[0]
        
x['cpu_type'] = x.apply(find_cpu_type, axis=1)
x[x['cpu_type'].isna()]

,instance_id,brand,cpu_brand,cpu_model,cpu_type,cpu_frequency,ram_capacity,ram_type,ram_frequency,hdd_capacity,ssd_capacity,weight,dimensions,title,new_title,new_title_tokens,screen_size,model
instance_id,,,,,,,,,,,,,,,,,,
www.isupplyhub.com//821,www.isupplyhub.com//821,hp,amd,None,None,None,4,ddr3 sdram. 4 gb sdram ddr3,None,500 gb,None,4.8 pounds,15.18 x 0.89 x 10.16 inches,amazon.com 15.6 hp 15-f009wm amd dual-core e1-...,amazon.com 15.6 hp 15f009wm amd dualcore e1210...,"[amazon.com, 15.6, hp, 15f009wm, amd, dualcore...",15.6,15f009wm amd


In [409]:
x.cpu_type.unique()

array(['i5', None, 'i7', 'a8', 'celeron', 'pentium', 'i3', 'a-series',
       'a4', 'e-series', 'e-300', 'e-450', 'e-350'], dtype=object)

In [344]:
# preprocess_laptop_dataset(x2.copy(deep=True)).head(20)

In [193]:
x2_dev = preprocess_laptop_dataset(x2.copy(deep=True))

In [80]:
x2_dev['tokens'] = x2_dev.title.apply(sp)

In [81]:
x2_dev.loc[:,['tokens']]

,tokens
0,"(lenovo, thinkpad, x230, 34352jf, tablet, pc, ..."
1,"(amazon.com, acer, aspire, v7, -, 582pg-6479, ..."
2,"(amazon.com, acer, aspire, e1, -, 572, -, 6870..."
3,"(amazon.com, 15.6, hp, 15, -, f009wm, amd, dua..."
4,"(amazon.com, asus, ux31a, -, xb52, 13.3, -, in..."
...,...
338,"(lenovo, thinkpad, x230, 2320, -, 12.5, -, cor..."
339,"(lenovo, thinkpad, x230, 2325, -, 12.5, -, cor..."
340,"(lenovo, thinkpad, x230, tablet, 3438, -, 12.5..."
341,"(lenovo, thinkpad, x230, 2324, -, 12.5, -, cor..."


In [109]:
x2_dev['new_title'] = x2_dev.title
irrelevant_regex = re.compile(r'[^a-z0-9.\s]')
multispace_regex = re.compile(r'\s\s+') # Why it doesn't work
x2_dev['new_title'] = x2_dev.new_title.str.lower().str.replace(irrelevant_regex, '').str.replace(multispace_regex, ' ')

In [110]:
x2_dev['model'] = pd.Series("" * len(x2_dev))

In [187]:
def get_laptop_model(record):
    brand_tokens = [w.text for w in sp(record['new_title'])]
    
    print(record['new_title'])
    
    screen_sizes = set(pd.read_csv('laptops.csv').Inches)
    screen_sizes = [str(s).lower() for s in screen_sizes]
    
    try:
        brand_index = brand_tokens.index(record['brand'])
        finish_index = brand_index + 1
        should_break = False
        for i in range(2 + brand_index, 5 + brand_index, 1):
            for sc in screen_sizes:
                if (sc in brand_tokens[i]):
                    should_break = True
                    break
            if should_break:
                break
            if not (brand_tokens[i].isalpha()):
                finish_index = i 
            else:
                break
                
    except:
        brand_index = -1
        
    if brand_index == -1:
        return ""
    
    return ' '.join(brand_tokens[brand_index+1:finish_index+1])
    

In [189]:
x2_dev

,instance_id,brand,cpu_brand,cpu_model,cpu_type,cpu_frequency,ram_capacity,ram_type,ram_frequency,hdd_capacity,ssd_capacity,weight,dimensions,title,tokens,new_title,model
0,www.softwarecity.ca//737,lenovo,intel,i5-3320m,dual-core 2 core . core i5,2.60 ghz,4,ddr3 sdram. ddr3-1600 pc3-12800. ddr3 sdram,ddr3-1600 pc3-12800,320 gb,-999,1.80 kg,-999,lenovo thinkpad x230 34352jf tablet pc - 12.5 ...,"(lenovo, thinkpad, x230, 34352jf, tablet, pc, ...",lenovo thinkpad x230 34352jf tablet pc 12.5 in...,thinkpad x230 34352jf
1,www.isupplyhub.com//1256,acer,intel,-999,1.6 ghz intel core i5-4200u,1.6 ghz intel core i5-4200u,8,ddr3 sdram. 8 gb ddr3l sdram,-999,500 gb mechanical hard drive,-999,4.8 pounds,15.02 x 10.08 x 0.90 inches,amazon.com acer aspire v7-582pg-6479 15.6-inch...,"(amazon.com, acer, aspire, v7, -, 582pg-6479, ...",amazon.com acer aspire v7582pg6479 15.6inch to...,aspire v7582pg6479
2,www.isupplyhub.com//326,acer,intel,-999,1.6 ghz intel core i5,1.6 ghz intel core i5,4,ddr3 sdram. 4 gb ddr3-sdram,-999,500 gb mechanical hard drive,-999,5.2 pounds,15.02 x 10.08 x 1 inches,amazon.com acer aspire e1-572-6870 15.6 inch l...,"(amazon.com, acer, aspire, e1, -, 572, -, 6870...",amazon.com acer aspire e15726870 15.6 inch lap...,aspire e15726870
3,www.isupplyhub.com//821,hp,amd,-999,-999,-999,4,ddr3 sdram. 4 gb sdram ddr3,-999,500 gb,-999,4.8 pounds,15.18 x 0.89 x 10.16 inches,amazon.com 15.6 hp 15-f009wm amd dual-core e1-...,"(amazon.com, 15.6, hp, 15, -, f009wm, amd, dua...",amazon.com 15.6 hp 15f009wm amd dualcore e1210...,15f009wm
4,www.isupplyhub.com//157,asus,intel,-999,1.7 ghz core i5-3317u,1.7 ghz core i5-3317u,4,ddr3 sdram. 4 gb ddr3,-999,256 mb,-999,2.9 pounds,8.80 x 0.70 x 12.80 inches,amazon.com asus ux31a-xb52 13.3-inch ultrabook...,"(amazon.com, asus, ux31a, -, xb52, 13.3, -, in...",amazon.com asus ux31axb52 13.3inch ultrabook 1...,ux31axb52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,www.vology.com//873,lenovo,intel,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,4,4 gb ddr3 slots qty 2 empty slots 1 max ram su...,4 gb ddr3 slots qty 2 empty slots 1 max ram su...,180 gb ssd. 180 gb ssd. lenovo thinkpad x230 2...,180 gb ssd. 180 gb ssd,3.3 lbs 3.3 lbs,8.1 in. 12 in x 8.1 in x 1 in. 1 in. 12 in,lenovo thinkpad x230 2320 - 12.5 - core i5 332...,"(lenovo, thinkpad, x230, 2320, -, 12.5, -, cor...",lenovo thinkpad x230 2320 12.5 core i5 3320m w...,thinkpad x230 2320
339,www.vology.com//823,lenovo,intel,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,4,4 gb ddr3 slots qty 2 max ram supported 16 gb ...,4 gb ddr3 slots qty 2 max ram supported 16 gb ...,500 gb hdd 7200 rpm. 500 gb hdd 7200 rpm. leno...,500 gb hdd 7200 rpm. 500 gb hdd 7200 rpm,3.3 lbs 3.3 lbs,8.1 in. 12 in x 8.1 in x 1 in. 1 in. 12 in,lenovo thinkpad x230 2325 - 12.5 - core i5 332...,"(lenovo, thinkpad, x230, 2325, -, 12.5, -, cor...",lenovo thinkpad x230 2325 12.5 core i5 3320m w...,thinkpad x230 2325
340,www.vology.com//2723,lenovo,intel,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,4,form factor so dimm 204-pin technology ddr3 sd...,form factor so dimm 204-pin technology ddr3 sd...,500 gb hdd 7200 rpm. 500 gb hdd 7200 rpm. leno...,500 gb hdd 7200 rpm. 500 gb hdd 7200 rpm,4 lbs 4 lbs,9 in. 12 in x 9 in x 1.2 in. 1.2 in. 12 in,lenovo thinkpad x230 tablet 3438 - 12.5 - core...,"(lenovo, thinkpad, x230, tablet, 3438, -, 12.5...",lenovo thinkpad x230 tablet 3438 12.5 core i5 ...,thinkpad x230
341,www.vology.com//1349,lenovo,intel,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,4,form factor so dimm 204-pin technology ddr3 sd...,fo

In [276]:
x4 = pd.read_csv('../data/sigmod/X4.csv')
x4.set_index('instance_id', inplace=True)
x4.columns

Index(['name', 'price', 'brand', 'size'], dtype='object')

In [277]:
x4.head()

,name,price,brand,size
instance_id,,,,
altosight.com//0,Lexar 32GB 1400x 210MB/s professional XQD memó...,31990.0,LEXAR,32 GB
altosight.com//25,Sony microSDXC UHS-1 U3 128GB memóriakártya + ...,59990.0,SONY,128 GB
altosight.com//66,"SANDISK DUAL DRIVE, TYPE-C, USB 3.0, 16GB, 130...",6790.0,SANDISK,16 GB
altosight.com//68,"SanDisk Dual Drive, TYPE-C, USB 3.0, 64GB, 150...",14190.0,SANDISK,64 GB
altosight.com//94,Lexar XQD 32GB X1400 PROFESSIONAL XQD +KUPON P...,329.0,LEXAR,32 GB


In [279]:
def preprocess_products_dataset(df):
    # Alpha numeric
    irrelevant_regex = re.compile(r'[^a-z0-9.\-\s]')
    multispace_regex = re.compile(r'\s\s+')

    for column in df.columns:
        if column == 'instance_id':
            continue
        df[column] = df[column].str.lower().str.replace(irrelevant_regex, ' ').str.replace(multispace_regex, ' ')
    
    return df



In [10]:
x4.info

<bound method DataFrame.info of                                                   name     price    brand  \
0    Lexar 32GB 1400x 210MB/s professional XQD memó...  31990.00    LEXAR   
1    Sony microSDXC UHS-1 U3 128GB memóriakártya + ...  59990.00     SONY   
2    SANDISK DUAL DRIVE, TYPE-C, USB 3.0, 16GB, 130...   6790.00  SANDISK   
3    SanDisk Dual Drive, TYPE-C, USB 3.0, 64GB, 150...  14190.00  SANDISK   
4    Lexar XQD 32GB X1400 PROFESSIONAL XQD +KUPON P...    329.00    LEXAR   
..                                                 ...       ...      ...   
830                 Pendrive LEXAR Jumpdrive S25 64 GB     24.99    LEXAR   
831              Pendrive SANDISK Extreme GO 3.1 256GB    195.99  SANDISK   
832        Tarjeta de Memoria SD  SONY 16GB EXPERIENCE     15.99     SONY   
833    Tarjeta de Memoria SANDISK Extreme Pro SDHC 8GB     22.97  SANDISK   
834                Tarjeta de Memoria SANDISK SDHC 8GB      4.99  SANDISK   

       size           instance_id  
0     3

In [45]:
x4_dev.head()

,name,price,brand,size,instance_id
0,Lexar 32GB 1400x 210MB/s professional XQD memó...,31990,LEXAR,32 GB,altosight.com//0
1,Sony microSDXC UHS-1 U3 128GB memóriakártya + ...,59990,SONY,128 GB,altosight.com//25
2,"SANDISK DUAL DRIVE, TYPE-C, USB 3.0, 16GB, 130...",6790,SANDISK,16 GB,altosight.com//66
3,"SanDisk Dual Drive, TYPE-C, USB 3.0, 64GB, 150...",14190,SANDISK,64 GB,altosight.com//68
4,Lexar XQD 32GB X1400 PROFESSIONAL XQD +KUPON P...,329,LEXAR,32 GB,altosight.com//94


In [572]:
with open('translations_lookup_all.json') as fin:
    variants = json.load(fin)

with open('langs_dict.json') as fin:
    json.load(fin)

In [576]:
# (x4_dev.sample(50, random_state=2459))

In [577]:
# list(x4_dev.name.sample(50, random_state=2459))

In [578]:
# Split into 3 tables
# ans = [pd.DataFrame(y) for x, y in x4_dev.groupby('product_type', as_index=False)]

In [579]:
# flash = None
# mobile = None
# stick = None
# tv = None
# ssd = None # TODO
# for df in ans:
#     if list(df.product_type)[0] == 'flash':
#         flash = df
#     elif list(df.product_type)[0] == 'mobile':
#         mobile = df
#     elif list(df.product_type)[0] == 'tv':
#         tv = df
#     else:
#         stick = df

In [580]:
# Translations
from deep_translator import GoogleTranslator
from collections import defaultdict

# variants = defaultdict(list)
# langs_dict = GoogleTranslator.get_supported_languages(as_dict=True)

In [581]:
def get_words_translations(word):
    words = []
    
    for dest_lang, l in langs_dict.items():
        if l == 'en':
            continue    
    
        words.append(GoogleTranslator(source='en', target=l).translate(word))
    
    return words

In [582]:
# # word: shop
# variants['shop'] = get_words_translations("shop")
# # word: storage
# variants['storage'] = get_words_translations('storage')
# # word: memory
# variants['memory'] = get_words_translations('memory')
# # word: card
# variants['card'] = get_words_translations('card')
# # word: buy
# variants['buy'] = get_words_translations('buy')

In [583]:
# import json
# with open('translations_lookup.json', 'w') as fout:
#     json.dump(variants, fout)

# with open('langs_dict.json', 'w') as fout:
#     json.dump(langs_dict, fout)

In [584]:

    
# x4_dev[column] = x4_dev[column].str.lower().str.replace(irrelevant_regex, ' ').str.replace(multispace_regex, ' ')

In [585]:
list(x4_dev.name.sample(50, random_state=2459))

[' usb ljdc20m 128bbeu ',
 ' microsdhc n',
 ' uhs class 3 class10 sdxc uhs i',
 'cl usb p20 usb 3.0 128 go 4187407 ',
 ' jumpdrive s25 usb 3.0 violeta fnac.es ',
 ' ultimate sdxc uhs ii u3',
 'usb datatraveler g4 fialov',
 ' usm32gr',
 'microsdhc extreme pro uhs 1 en fnac.es',
 'tesco direct jumpdrive c20m usb 3.0 3.1 gen 1 type a black silver',
 'cl usb jumpdrive c20c usb 3.0 3.1 gen 1 type a type c noir argent lecteur usb mk1008030400 ',
 ' galaxy a50 dual sim 128 gb 4g lte white',
 ' premium line usb 3.0 3534490 ',
 ' ultra plus sdxc 64go 80mo seconde uhs i n',
 ' ultra plus class 10 sdhc ',
 'drive cruzer extreme usb3.0 ',
 ' exceria pro n401 sd 95 mb s 4k hd thn n401s0640e4',
 ' basic usb2.0 pendrive usb mercado actual ',
 'usb jumpdrive c20c en fnac.es',
 ' 3502450 rainbow usb 2.0 drive transparent blue',
 ' fd128att4 ef attache usb 2.0 drive',
 ' sdcz880 256g g46 cruzer extreme usb portatile',
 ' 300x premium ii sdhc',
 ' premium line usb usb 3.0 ',
 ' scheda sd professionale sd

In [586]:
list(x4.name.sample(50, random_state=2459))

['LEXAR USB-stick 128 GB (LJDC20M-128BBEU)',
 'SanDisk - flashgeheugenkaart - 4 GB - microSDHC\\n',
 'Kingston - Flash memory card - 128 GB - UHS Class 3 / Class10 - SDXC UHS-I',
 'Clé USB Lexar P20 USB 3.0 128 GO (4187407)',
 'Lexar JumpDrive S25 Memoria USB 3.0 64 GB violeta Fnac.es Lexar',
 'Transcend 64 GB Ultimate SDXC UHS-II U3 Memory Card',
 'USB Flash Kingston DataTraveler G4 64GB fialový',
 'Sony USM32GR',
 'Tarjeta microSDHC SanDisk Extreme Pro UHS-1 16GB en Fnac.es',
 'Tesco direct: Lexar JumpDrive C20m 128GB USB 3.0 (3.1 Gen 1) Type-A Black Silver',
 'Clé USB Lexar JumpDrive C20c 128GB 128GB USB 3.0 (3.1 Gen 1) Type-A/Type-C Noir, Argent lecteur USB flash (MK1008030400)',
 'Samsung Galaxy A50 Dual Sim, 128 GB, 4G LTE, White',
 'Intenso premium line 64gb usb 3.0 (3534490)',
 'Carte Mémoire Sandisk ultra PLUS SDXC 64Go 80Mo/seconde UHS-I\\n',
 'SANDISK Ultra Plus Class 10 SDHC Memory Card - 32 GB',
 'FLASH DRIVE, CRUZER EXTREME USB3.0 16GB',
 'Toshiba Exceria Pro N401 64GB SD

In [462]:
re.sub('\d\d\s?gb', '', ' usb-stick 128 gb ljdc20m-128bbeu ', 6)
re.sub('\d\d\s?gb', '', ' usb-stick 128 gb ljdc20m-128bbeu ', 6)
re.sub('\d\d\s?gb', '', ' usb-stick 128 gb ljdc20m-128bbeu ', 6)

' usb-stick 128 gb ljdc20m-128bbeu '

In [587]:
def get_type(record):
    name = record['name'].lower()
    
    if pd.isna(record['size']):
        if 'tv' in name:
            return 'tv'
        return 'mobile'
    
    
    flash_keywords = ['usb', 'drive']
    memory_stick_keywords = ['card', 'stick', 'sd', 'microsd', 'hc', 'class', 'speicherkarte'] # Add variants here
    
    is_flash = False
    is_memory = False
    
    for w in flash_keywords:
        if w in name:
            is_flash = True
            break
            
    for w in memory_stick_keywords:
        if w in name:
            is_memory = True
            break
    
    if is_flash:
        return 'flash'
    
    if is_memory:
        return 'stick'
    
    return 'stick'

def convert_numbers_to_strings(df, cols_to_convert, remove_point_zero=True):
    """ Convert number types to strings in a dataframe.
        This is convoluted as need to keep NoneTypes as NoneTypes for what comes next!
        
        Inputs: - df -> dataframe to convert number types
                - cols_to_convert -> list of columns to convert
                - remove_point_zero -> bool to say whether you want '.0' removed from number
        
        Ouputs: - dataframe with converted number types
    """
    new_df = df.copy()
    for col in cols_to_convert:
        if remove_point_zero:
            new_df[col] = new_df[col].apply(lambda x: str(x).replace('.0','')\
                                            if not isinstance(x, type(None)) else x)
        else:
            new_df[col] = new_df[col].apply(lambda x: str(x)\
                                            if not isinstance(x, type(None)) else x)
    return new_df


def preprocess_products_dataset(x4_dev):
    # Clean x4
    # Alpha numeric
    irrelevant_regex = re.compile(r'[^a-z0-9,.\-\s]')
    multispace_regex = re.compile(r'\s\s+') # Why it doesn't work
    x4_dev.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

    for column in x4_dev.columns:
        if column == 'instance_id':
            continue
        x4_dev[column] = x4_dev[column].str.lower().str.replace(irrelevant_regex, ' ').str.replace(multispace_regex, ' ')
        
    x4_dev['product_type'] = x4_dev.apply(get_type, axis=1)
    x4_dev.drop('price', inplace=True, axis=1)
    x4_dev['size'] = x4_dev['size'].str.lower().str.replace(' ', '')
    x4_dev['size'] = x4_dev['size'].where(x4_dev['size'].notnull(), 0)
    
    # Remove unwanted words from the name
    for i in range(len(x4_dev)):
        record = x4_dev.iloc[i]

        name = record['name']

        # remove unnecessary characters
        basic_punct = '-/\*_,:;/()®™' 
        punct_to_space = str.maketrans(basic_punct, ' ' * len(basic_punct))  # map punctuation to space
        name = name.translate(punct_to_space)

        # remove brand
        name = name.replace(record['brand'], '')

        # remove size

        if record.product_type in ['flash', 'stick']:
            name = re.sub('\d\d\d\s?gb', '', name, 6)
            name = re.sub('\d\d\s?gb', '', name, 6)
            name = re.sub('\d\s?gb', '', name, 6)

        tokens = name.split(' ')
        for wd, wdtl in variants.items():
            while wd in tokens:
                tokens.remove(wd)
            for wdt in wdtl:
                while wdt in tokens:
                    tokens.remove(wdt) 

        unneeded_words = ['mmoire', 'speicherkarte', 'flashgeheugenkaart', 'flash', 'stick', 'speed', 'high']
        for w in unneeded_words:
            while w in tokens:
                tokens.remove(w)
        x4_dev.iloc[i]['name'] = ' '.join(tokens)

    for column in x4_dev.columns:
        if column == 'instance_id':
            continue
        x4_dev[column] = x4_dev[column].str.lower().str.replace(irrelevant_regex, ' ').str.replace(multispace_regex, ' ')
        
    return x4_dev


In [588]:
x4_dev = convert_numbers_to_strings(x4, ['price']).copy(deep=True)

In [590]:
x4_dev = preprocess_products_dataset(x4_dev)

In [591]:
x4_dev

,name,brand,size,instance_id,product_type
0,1400x 210mb s professional xqd memriakrtya,lexar,32gb,altosight.com//0,stick
1,microsdxc uhs 1 u3 memriakrtya adapter srg1uxa,sony,128gb,altosight.com//25,stick
2,dual drive type c usb 3.0 130 mb s,sandisk,16gb,altosight.com//66,flash
3,dual drive type c usb 3.0 150 mb s,sandisk,64gb,altosight.com//68,flash
4,xqd x1400 professional xqd kupon premium 50 n...,lexar,32gb,altosight.com//94,stick
...,...,...,...,...,...
830,pendrive jumpdrive s25,lexar,64gb,altosight.com//13815,flash
831,pendrive extreme go 3.1,sandisk,256gb,altosight.com//13832,flash
832,de sd experience,sony,16gb,altosight.com//13932,stick
833,de extreme pro sdhc,sandisk,8gb,altosight.com//13944,stick


In [592]:
list(x4_dev.name.sample(50, random_state=2459))

[' usb ljdc20m 128bbeu ',
 ' microsdhc n',
 ' uhs class 3 class10 sdxc uhs i',
 'cl usb p20 usb 3.0 128 go 4187407 ',
 ' jumpdrive s25 usb 3.0 violeta fnac.es ',
 ' ultimate sdxc uhs ii u3',
 'usb datatraveler g4 fialov',
 ' usm32gr',
 'microsdhc extreme pro uhs 1 en fnac.es',
 'tesco direct jumpdrive c20m usb 3.0 3.1 gen 1 type a black silver',
 'cl usb jumpdrive c20c usb 3.0 3.1 gen 1 type a type c noir argent lecteur usb mk1008030400 ',
 ' galaxy a50 dual sim 128 gb 4g lte white',
 ' premium line usb 3.0 3534490 ',
 ' ultra plus sdxc 64go 80mo seconde uhs i n',
 ' ultra plus class 10 sdhc ',
 'drive cruzer extreme usb3.0 ',
 ' exceria pro n401 sd 95 mb s 4k hd thn n401s0640e4',
 ' basic usb2.0 pendrive usb mercado actual ',
 'usb jumpdrive c20c en fnac.es',
 ' 3502450 rainbow usb 2.0 drive transparent blue',
 ' fd128att4 ef attache usb 2.0 drive',
 ' sdcz880 256g g46 cruzer extreme usb portatile',
 ' 300x premium ii sdhc',
 ' premium line usb usb 3.0 ',
 ' scheda sd professionale sd